This notebook runs through an implementation of image classification using Keras. The dataset being used is the Blood Cells dataset. The goal is to create a Convolutional Neural Network that can classify differet forms of blood cells. This notebook demonstrates an example of finetuning a neural network. The premade VGG19 architecture and weights are used, but several of the layers are unfrozen and trained. In addition, several new layers are added to the architecture.

The dataset comes with a subset of images that have been augmented and contain additional label subtypes. It uses the "Dataset2-master" folder, whihc has 2500 images (augmented from the original images. The four cell types are: Neutrophil, Eosinophil, Monocyte, and Lymphocyte. (Some cell images in the database have more than one nucleus, so they have two labels instead of one, but for the moment we are concerned only with classifying the four primary classes. Just keep in mind accuracy would improve if we accounted for these.)

The accuracy demonstrated in this notbeook won't be that great, but this notebook is primarily concerned with demonstrating how to implement a pretrained network and fine-tune it. Try playing with the model and training arguments to see what kind of accuracy you can get from it. Try varying image size by shifting the target_size and input_shape paramaters. The larger the images being put into the model, the more the model will learn, up to the total size of the image.

To begin with, we will start by importing all the libraries we need.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications import VGG19

We'll be using the built-in Keras ImageDataGenerator to get the image data from our directory. It's a handy function that can collect and sort image data easily, saving a lot of manual preprocessing and data handling. We need to set the directories we are drawing the trainign data from, and then we need to create instances of the ImageDataGenerator. We'll also specify some arguments for later use, like the batch size and number of classes here.

In [ ]:
train_dataset_path = "/images/TRAIN/"
test_dataset_path = "/images/TEST/"
pred_dataset_path = "/images/TEST_SIMPLE/"

list_dir = os.listdir(train_dataset_path)
num_classes = len(list_dir)
batch = 16
classes = ['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

We'll now use the function to go through all the folders in the training and testing folders and separating the images into features and labels.

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dataset_path,
        target_size=(240, 320),
        batch_size=batch,
        shuffle=True,
        seed=None,
        class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
        test_dataset_path,
        target_size=(240, 320),
        batch_size=batch,
        shuffle=True,
        seed=None,
        class_mode="categorical")

Now we can start defining our model. In this example, we'll be using the premade VGG19 model and finetuning it to our needs. So after importing it, we first declare an instance of the model, along with what weights we want to use, and the input shape that the image should be in.

In [ ]:
vgg_conv = VGG19(weights='imagenet',
                 include_top=False,
                 input_shape=(240, 320, 3))

Now we'll make certain layers trainable. When using a preestablished neural network architecture, by default the layers in the architecture are "frozen" meaning they won't be trained. Frequently networks are used like this and a final classification layer is added to the network to fit the dataset currently being worked on. It is also common for certain layers to be "unfrozen", or made trainable. It is quite easy to make layers trainable, you just need to declare that the layers you want are unfrozen. 

We'll declare that everything but the last five layers of the model aren't trainable, which makes the last five trainable. We can confirm this by printing just the trainable layers.

In [ ]:
for layer in vgg_conv.layers[:-14]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

Beyond just making some layers in the network trainable (unfreeezing them), we can also add layers to the base model. This is done by simply declaring a new model architecture with the `sequential` function, adding in the pretrained model, and then adding new layers on top of that. We can then be sure that the new model contains both the premade layers and the new layers we've specified by printing the `summary()` of the model.

In [ ]:
new_model = Sequential()
new_model.add(vgg_conv)
new_model.add(Flatten())
new_model.add(BatchNormalization())
new_model.add(Dense(256, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(BatchNormalization())
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(BatchNormalization())
new_model.add(Dense(64, activation='relu'))
new_model.add(BatchNormalization())
new_model.add(Dropout(0.2))
new_model.add(Dense(len(classes), activation='softmax'))
print(new_model.summary())
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Before we fit the model, we'll define some callbacks. We want to have a checkpoint system in place, creating checkpoints when performance on the validation set improves. We'll also be adjusting the learning rate for the classifier, reducing it whenever it hits a pleateu, and we use the `ReduceLROnPleateu` function for this. Finally, we'll enable early stopiing, allowing the model to stop training early if it stops making significant progress.

In [ ]:
filepath = "weights_vgg19.hdf5"
callbacks = [ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'),
              ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='min', min_lr=0.00001),
              EarlyStopping(monitor= 'val_loss', min_delta=1e-10, patience=15, verbose=1, restore_best_weights=True)]

Now we can fit the model with our chosen arguments, including our callbacks, chosen number of training epochs, and the test data as the validation data.

In [ ]:
# try "train_generator.n // train_generator.batch_size " for steps per epoch and validation steps
train_records = new_model.fit_generator(train_generator,
         epochs = 80,
         steps_per_epoch= 100,
         validation_data = test_generator,
         validation_steps= 100,
         callbacks = callbacks,
         verbose = 1)

Now we want to evaluate the performance of our classifier. Let's visualize how the loss and accuarcy change over time on the validation and training sets. First, we're going to select what variables/statistics we are interested in analyzing. This will be the accuracy and the loss for both the training and validation sets. We'll also be needing the length of time the model was trained for, so we'll get the length of the training accuracy.

In [ ]:
# visualize training and validation accuracy/loss
# declare important variables
training_acc = train_records.history['acc']
val_acc = train_records.history['val_acc']
training_loss = train_records.history['loss']
validation_loss = train_records.history['val_loss']

# gets the length of how long the model was trained for
num_epochs = range(1, len(training_acc) + 1)

Now we ought to be able to plot the loss and accuracy across the number of epochs we were training for. First, let's plot the loss.

In [ ]:
# plot the loss across the number of epochs
plt.figure()
plt.plot(num_epochs, training_loss, label='Training Loss')
plt.plot(num_epochs, validation_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

Now we'll plot the accuracy.

In [ ]:
plt.figure()
plt.plot(num_epochs, training_acc, label='Training Accuracy')
plt.plot(num_epochs, val_acc, label='Training Accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Finally, let's print the accuracy.

In [ ]:
# evaluate generator
score = new_model.evaluate_generator(test_generator, verbose=0, steps=32)
print('\nAchieved Accuracy:', score[1],'\n')

There we go, we've implemented a fine-tuned CNN on a dataset, and then checked how the CNN performed, all with relatively few lines of code.